# Qd-Sätze schreiben

In [1]:
import pandas as pd
import re
from datetime import datetime

rawpath = '../raw'
datpath = '../dat'

Aktuelle csv-Datei wird als df importiert, mit convert_dtypes werden die neuen String-Formate von pandas 1.0 angewendet.

Die Konkordanz wird als tp_idns geladen.

In [2]:
df = pd.read_csv(f'{rawpath}/HABV_97_1_v2.csv').convert_dtypes()
tp_idns = pd.read_csv(f'{rawpath}/koko_mabv_appr_20211116.csv',delimiter='\t', encoding='utf-8')
tp_idns['id'] = tp_idns['akten-nr'].str.slice(10).str.strip()
tp_idns.set_index('id', inplace=True)

Die Daten werden als Pica+ in eine .dat-Datei geschrieben

In [3]:
def pica3_schreiben(row):
    pica = """\t\n0500 Qd
0501 Text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
    if pd.isna(row.lz_ende):
        pica += f"""1100 {row.lz_beginn}\n1110 *{row.lz_beginn}$4lfzt"""
    elif pd.notna(row.lz_ende):
        pica += f"""1100 {row.lz_beginn}$b{row.lz_ende}\n1110 *{row.lz_beginn}-{row.lz_ende}$4lfzt"""
    pica += """
1130 TB-papier
1130 !040445224!
1130 !041466098!
1132 a1-analog;a2-masch;f1-text;f2-blatt;e2-uv;v-cont
1500 /1ger
1700 /1XA-DE
4000 $aMitgliedsakte """
    if pd.notna(row.Titel):
        pica +=  f'{row.Titel} '
    pica +=  f'{row.Vorname.strip()} {row.Nachname.strip()}'
    if pd.notna(row.Zusaetze):
        pica +=  f', {row.Zusaetze}'
    pica += f"""
4019 Mitgliedsakte$Bobja
3100 !000073369!$BZusammenstellender$4com"""
    if pd.notna(row.Ort):
        pica += f"\n4046 {row.Ort}"
    pica += """
4060 1 Akte
4105 !1061153886!
5100 !000073369!
"""
    try:
        konko_idn = tp_idns.loc[f'{row.Id}'].idn.strip()
    except:
        konko_idn = None
    
    if konko_idn is not None:
        pica += f"5101 !{konko_idn}!\n"

    elif pd.notna(row.idns):
        if idn := re.match(r"\[\'(\w+)\'\]", row.idns):
            pica += f"5101 !{idn.group(1)}!\n"
        else:
            pica += f"5550 [DBSM-S]{row.Vorname.strip()} {row.Nachname.strip()}\n"
            
    else:
        pica += f"5550 [DBSM-S]{row.Vorname.strip()} {row.Nachname.strip()}\n"
    pica += "5109 (DE-101d){DE-101d}"

    if pd.notna(row.firmen_idns):
        if firmen_idn := re.match(r'\[\'(w+)\'\]', row.firmen_idns):
            pica += f"5110 {firmen_idn.group(1)}"
    pica += f"""
5560 [RNA]Korrespondenz
5590 [Objektgattung]
5590 !040009254!
4700 |BSM|
7001 {now.strftime('%d-%m-%y')} : x
7100 HA/BV 97.1,{row.Id} @ m
7109 !!DBSM/S!! ; HA/BV 97.1,{row.Id}
6710 !1059758504!$l{row.Id}\n"""
    return pica

In [4]:
# jede einzelne zeile des df wird ausgewertet und das ergebnis als string pica zurückgegeben
# es werden immer drei dateien geschrieben: eine -recent.dat mit allen datensätzen, eine backup-datei mit aktuellem datumsstempel und eine datei mit 5 zufällig ausgewählten datensätzen
now = datetime.now()
with open(f"{datpath}/97_1_pica3-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat", 'w') as f:
    with open(f"{datpath}/97_1_pica3-recent.dat", 'w') as f2:
        for row in df.itertuples():
            f.write(pica3_schreiben(row))
            f2.write(pica3_schreiben(row))

with open(f"{datpath}/97_1_pica3-recent-sample.dat", 'w') as f:
    for row in df.sample(5).itertuples():
        f.write(pica3_schreiben(row))

# Personen-Sätze schreiben

In [4]:
def tp_schreiben(row):
    pica = f"""
001U $0utf8
001X $00
002@ $0Tp4
004B $apiz
008A $af
008B $ao
028A $d{row.Vorname.strip()}$a{row.Nachname.strip()}
029R $9000073369$4affi$vMitglied
"""
    if pd.notna(row.firmen_idns):
        firmen_idn = re.match(r'\[\'(.+)\'\]', row.firmen_idns).group(1)
        pica += f'029R $9{firmen_idn}$4affi\n'
    if pd.notna(row.Geschlecht) and row.Geschlecht == 'w':
        pica += f"032T $af\n"
    elif pd.notna(row.Geschlecht) and row.Geschlecht == 'm':
        pica += f"032T $am\n"
    if pd.notna(row.Titel):
        pica += f'041R $a{row.Titel}$4akad\n'
    if pd.notna(row.firmenart):
        if row.firmenart == 'Sortiment':
            if pd.notna(row.Geschlecht) and row.Geschlecht == 'f':
                #Buchhändlerin
                pica += f'041R $9942316746$4berc'
            elif pd.notna(row.Geschlecht) and row.Geschlecht == 'm':
                #Buchhändler
                pica += f'041R $9041309480$4berc'
            else:
                #Buchhändler
                pica += f'041R $9041309480$4berc'
        elif row.firmenart == "Verlag":
            if row.Geschlecht == 'f':
                #verlegerin
                pica += f'041R $9042667208$4berc'
            elif row.Geschlecht == 'm':
                #verleger
                pica += f'041R $9041383435$4berc'
            else:
                #verleger
                pica += f'041R $9041383435$4berc'
        elif row.firmenart == 'Zwischenbuchhandel':
            if row.Geschlecht == 'f':
                #Buchhändlerin
                pica += f'041R $9942316746$4berc'
            elif row.Geschlecht == 'm':
                #Buchhändler
                pica += f'041R $9041309480$4berc'
            else:
                #Buchhändler
                pica += f'041R $9041309480$4berc'
        elif row.firmenart == 'Verlagsvertreter':
            if row.Geschlecht == 'f':
                #Verlagsvertreterin 1205182039
                pica += f'041R $91205182039$4berc'
            elif row.Geschlecht == 'm':
                #Verlagsvertreter 041346645
                pica += f'041R $9041346645$4berc'
            else:
                #Verlagsvertreter 041346645
                pica += f'041R $9041346645$4berc'
    pica += """
042B $aXA-DE
047A/03 $eDE-101d
047A/03 $rDE-101
050C $aMitgliedsakte des Börsenvereins des Deutschen Buchhandels
"""
    if pd.notna(row.tod): #sterbedatum ist bekannt
        if pd.notna(row.geburt): #gebdatum ist bekannt
            #datl geb+tod
            pica += f'060R $a{row.geburt[-4:]}$b{row.tod[-4:]}$4datl\n'
            #datx geb+tod
            pica += f'060R $a{row.geburt}$b{row.tod}$4datx'
        elif pd.isna(row.geburt):
            #datl tod
            pica += f'060R $b{row.tod[-4:]}$4datl\n'
            #datx tod
            pica += f'060R $b{row.tod}$4datx'
    elif pd.isna(row.tod): #kein sterbedatum bekannt
        if pd.notna(row.geburt): #gebdatum ist bekannt
            #nur datl geb
            pica += f'060R $a{row.geburt[-4:]}$4datl'
    if pd.notna(row.Ort):
        pica += f'\n065R $a{row.Ort}$4ortw' 
    pica += f"""\n070A/03 $SDE-101d$0HABV_91_1_{row.Id}\n"""
    return pica

filt = (df['idns'].isna() & df['doppelname'].isna() & df['geburt'].notna())

with open('test-pica+-tp-2021-08-30-korr.dat', 'w') as f:
    for index, row in df[filt].iterrows():
        f.write(tp_schreiben(row))

In [4]:
def tp_pica3_schreiben(row):
    pica = f"""
005 Tp4
008 piz
011 f
012 o
043 XA-DE
100 {row.Nachname.strip()}, {row.Vorname.strip()}
510 !000073369!$4affi$vMitglied
"""
    if pd.notna(row.firmen_idns):
        firmen_idn = re.match(r'\[\'(.+)\'\]', row.firmen_idns).group(1)
        pica += f'510 !{firmen_idn}!$4affi\n'
    if pd.notna(row.Geschlecht) and row.Geschlecht == 'w':
        pica += f'375 f\n'
    elif pd.notna(row.Geschlecht) and row.Geschlecht == 'm':
        pica += f'375 m\n'
    if pd.notna(row.Titel):
        pica += f'550 {row.Titel}$4akad\n'
    if pd.notna(row.firmenart):
        if row.firmenart == 'Sortiment':
            if pd.notna(row.Geschlecht) and row.Geschlecht == 'w':
                #Buchhändlerin
                pica += f'550 !942316746!$4berc'
            elif pd.notna(row.Geschlecht) and row.Geschlecht == 'm':
                #Buchhändler
                pica += f'550 !041309480!$4berc'
            else:
                #Buchhändler
                pica += f'550 !041309480!$4berc'
        elif row.firmenart == "Verlag":
            if row.Geschlecht == 'w':
                #verlegerin
                pica += f'550 !042667208!$4berc'
            elif row.Geschlecht == 'm':
                #verleger
                pica += f'550 !041383435!$4berc'
            else:
                #verleger
                pica += f'550 !041383435!$4berc'
        elif row.firmenart == 'Zwischenbuchhandel':
            if row.Geschlecht == 'w':
                #Buchhändlerin
                pica += f'550 !942316746!$4berc'
            elif row.Geschlecht == 'm':
                #Buchhändler
                pica += f'550 !041309480!$4berc'
            else:
                #Buchhändler
                pica += f'550 !041309480!$4berc'
        elif row.firmenart == 'Verlagsvertreter':
            if row.Geschlecht == 'w':
                #Verlagsvertreterin 1205182039
                pica += f'550 !1205182039!$4berc'
            elif row.Geschlecht == 'm':
                #Verlagsvertreter 041346645
                pica += f'550 !041346645!$4berc'
            else:
                #Verlagsvertreter 041346645
                pica += f'550 !041346645!$4berc'
    if pd.notna(row.tod): #sterbedatum ist bekannt
        if pd.notna(row.geburt): #gebdatum ist bekannt
            #datl geb+tod
            pica += f'548 {row.geburt[-4:]}$b{row.tod[-4:]}$4datl\n'
            #datx geb+tod
            pica += f'548 {row.geburt}$b{row.tod}$4datx'
        elif pd.isna(row.geburt):
            #datl tod
            pica += f'548 $b{row.tod[-4:]}$4datl\n'
            #datx tod
            pica += f'548 $b{row.tod}$4datx'
    elif pd.isna(row.tod): #kein sterbedatum bekannt
        if pd.notna(row.geburt): #gebdatum ist bekannt
            #nur datl geb
            pica += f'548 {row.geburt[-4:]}$4datl'
    if pd.notna(row.Ort):
        pica += f'\n551 {row.Ort}$4ortw'
    pica += f"""
903 $eDE-101d
903 $rDE-101
667 Mitgliedsakte des Börsenvereins des Deutschen Buchhandels
983 {row.Id}\n
"""
    return pica

In [8]:
# filter idns ist leer (d.h. es gibt keine gnd-matches), keine doppelnamen, wenigstens geburtsjahr ist bekannt.
filt = (df['idns'].isna() & df['doppelname'].isna() & df['geburt'].notna())

with open('97_1_pica3-2021-08-30.dat', 'w') as f:
    for row in df[filt].itertuples():
        f.write(tp_pica3_schreiben(row))

In [3]:
df

,Id,Name,Titel,Vorname,Nachname,Ort,Zusaetze,Lebensdaten,lz_beginn,lz_ende,Geschlecht,matches,idns,doppelname,firmen_matches,firmen_idns,firmenart,geburt,tod
0,1,"Abts-Kröger, Christel",<NA>,Christel,Abts-Kröger,Düsseldorf,Buchhandlung Emil Bierbaum,08.01.1913-,1955,1966,w,"[{'idn': '1204494460', 'datum_match': True, 'b...",['1204494460'],<NA>,"[{'idn': '2006064-6', 'level': 'gnd1', 'ort_ma...",['2006064-6'],Sortiment,<NA>,<NA>
1,2,"Achterberg, Georg",<NA>,Georg,Achterberg,Berlin,Eva Achterberg,<NA>,1955,1956,m,<NA>,<NA>,<NA>,<NA>,<NA>,Verlag,08.01.1913,<NA>
2,3,"Ackermann, Anna",<NA>,Anna,Ackermann,Stuttgart,<NA>,<NA>,1955,1956,w,<NA>,<NA>,<NA>,<NA>,<NA>,Verlag,<NA>,<NA>
3,4,"Ackermann, Eduard",<NA>,Eduard,Ackermann,Lübeck,<NA>,<NA>,1955,1965,m,<NA>,<NA>,<NA>,<NA>,<NA>,Sortiment,<NA>,<NA>
4,5,"Ackermann, Elisabeth",<NA>,Elisabeth,Ackermann,Gailingen,Gailinger Buchhandlung und Verlag Frau Mariann...,EA 18.12.1911-,1964,1968,w,<NA>,<NA>,<NA>,<NA>,<NA>,Sortiment,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2879,2880,"Zufall, Gerda",<NA>,Gerda,Zufall,Kassel,Buchhandlung Freyschmidt; Nachfolge: Dr. Hans ...,gestorben am 02.06.1963,1955,1963,w,<NA>,<NA>,<NA>,<NA>,<NA>,Sortiment,<NA>,<NA>
2880,2881,"Zulauf, Hugo",<NA>,Hugo,Zulauf,Mainz,Grünewald Verlag,23.06.1912-,1958,1965,m,<NA>,<NA>,<NA>,<NA>,<NA>,Verlag,<NA>,<NA>
2881,2882,"Zumstein, Anton",<NA>,Anton,Zumstein,München,Zumsteins Landkartenhaus; Nachfolge: Max Zumstein,18.06.1875-,1955,1965,m,<NA>,<NA>,<NA>,<NA>,<NA>,Zwischenbuchhandel,23.06.1912,<NA>
2882,2883,"Zutavern, Walter",<NA>,Walter,Zutavern,Pforzheim,W. Zutavern,04.02.1903-,1955,1965,m,<NA>,<NA>,<NA>,<NA>,<NA>,Sortiment,18.06.1875,<NA>


# Konkordanz-Merge

In [20]:

tp_idns

,idn,akten-nr
id,,
1634,1012288978,HABV_91_1_1634
1749,1012291685,HABV_91_1_1749
1176,1012562379,HABV_91_1_1176
455,1034719009,HABV_91_1_455
1477,104378425X,HABV_91_1_1477
...,...,...
2883,1280506490,HABV_91_1_2883
2588,128134704,HABV_91_1_2588
2099,129414468,HABV_91_1_2099


In [24]:
# return row of index 455 from tp_idns
tp_idns.loc['455'].idn

'1034719009                         '

In [36]:
tp_idns.loc[f'455'].idn.strip()

'1034719009'